# RME WY2004

A baseline isnobal run was performed using the original IPW version. Read in the ipw images and save to netcdf files.

Creates the following files:

- topo.nc with just the DEM
- converts the `snow.*` to `snow.nc`
- converts the `em.*` to `em.nc`
- converts the `in.*` into the input variables
- converts the `ppt.4b*` into the precipitation variables

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt

import pandas as pd
import netCDF4 as nc
from spatialnc.ipw import IPW

In [ ]:
# get the times for all the IPW output files
path_output = '../pysnobal/tests/test_data_spatial/gold_ipw/rme_wy2004/netcdf'
out_dir = '../pysnobal/tests/test_data_spatial/gold_ipw/rme_wy2004/output'
start_date = pd.to_datetime('10-01-2004 00:00:00-07:00')
time_zone = 'MST'

# x and y for the domain
init = IPW('../pysnobal/tests/test_data_spatial/gold_ipw/rme_wy2004/init.ipw')
x = init.bands[0].x
y = init.bands[0].y
nx = len(x)
ny = len(y)




In [ ]:
# Create the topo file
# This is not like the normal topo files that are used with AWSM and SMRF
dimensions = ('y', 'x')
topo = nc.Dataset(os.path.join(path_output, 'topo.nc'), 'w', clobber=True)

# create the dimensions
topo.createDimension('y', ny)
topo.createDimension('x', nx)

# create some variables
topo.createVariable('y', 'f', dimensions[0])
topo.createVariable('x', 'f', dimensions[1])

topo.variables['x'][:] = x
topo.variables['y'][:] = y

# topo image
topo.createVariable('dem', 'f', dimensions)
setattr(topo.variables['dem'], 'units', 'meters')
topo.variables['dem'] = init.bands[0].data

topo.close()


In [ ]:
# Convert all .ipw output files to netcdf files

# ========================================================================
# NetCDF EM image
# ========================================================================
m = {}
m['name'] = ['net_rad', 'sensible_heat', 'latent_heat', 'snow_soil',
                'precip_advected', 'sum_EB', 'evaporation', 'snowmelt',
                'SWI', 'cold_content']
m['units'] = ['W m-2', 'W m-2', 'W m-2', 'W m-2', 'W m-2', 'W m-2',
                'kg m-2', 'kg m-2', 'kg or mm m-2', 'J m-2']
m['description'] = ['Average net all-wave radiation',
                    'Average sensible heat transfer',
                    'Average latent heat exchange',
                    'Average snow/soil heat exchange',
                    'Average advected heat from precipitation',
                    'Average sum of EB terms for snowcover',
                    'Total evaporation',
                    'Total snowmelt',
                    'Total runoff',
                    'Snowcover cold content']

netcdfFile = os.path.join(path_output, 'em.nc')
os.remove(netcdfFile)
print(netcdfFile)

dimensions = ('time', 'y', 'x')
em = nc.Dataset(netcdfFile, 'w', clobber=True)

# create the dimensions
em.createDimension('time', None)
em.createDimension('y', ny)
em.createDimension('x', nx)

# create some variables
em.createVariable('time', 'f', dimensions[0])
em.createVariable('y', 'f', dimensions[1])
em.createVariable('x', 'f', dimensions[2])

setattr(em.variables['time'], 'units', 'hours since %s' % start_date)
setattr(em.variables['time'], 'calendar', 'standard')
setattr(em.variables['time'], 'time_zone', time_zone)
em.variables['x'][:] = x
em.variables['y'][:] = y

# em image
for i, v in enumerate(m['name']):
    em.createVariable(v, 'f', dimensions[:3], chunksizes=(24, 10, 10))
    setattr(em.variables[v], 'units', m['units'][i])
    setattr(em.variables[v], 'description', m['description'][i])

In [ ]:
# ========================================================================
# NetCDF SNOW image
# ========================================================================

s = {}
s['name'] = ['thickness', 'snow_density', 'specific_mass', 'liquid_water',
                'temp_surf', 'temp_lower', 'temp_snowcover',
                'thickness_lower', 'water_saturation']
s['units'] = ['m', 'kg m-3', 'kg m-2', 'kg m-2',
                'C', 'C', 'C', 'm', 'percent']
s['description'] = ['Predicted thickness of the snowcover',
                    'Predicted average snow density',
                    'Predicted specific mass of the snowcover',
                    'Predicted mass of liquid water in the snowcover',
                    'Predicted temperature of the surface layer',
                    'Predicted temperature of the lower layer',
                    'Predicted temperature of the snowcover',
                    'Predicted thickness of the lower layer',
                    'Predicted percentage of liquid water'
                    ' saturation of the snowcover']

netcdfFile = os.path.join(path_output, 'snow.nc')
os.remove(netcdfFile)
print(netcdfFile)

dimensions = ('time', 'y', 'x')
snow = nc.Dataset(netcdfFile, 'w', clobber=True)

# create the dimensions
snow.createDimension('time', None)
snow.createDimension('y', ny)
snow.createDimension('x', nx)

# create some variables
snow.createVariable('time', 'f', dimensions[0])
snow.createVariable('y', 'f', dimensions[1])
snow.createVariable('x', 'f', dimensions[2])

setattr(snow.variables['time'], 'units',
        'hours since %s' % start_date)
setattr(snow.variables['time'], 'calendar', 'standard')
setattr(snow.variables['time'], 'time_zone', time_zone)
snow.variables['x'][:] = x
snow.variables['y'][:] = y

for i, v in enumerate(s['name']):

    snow.createVariable(v, 'f', dimensions[:3], chunksizes=(6, 10, 10))
    setattr(snow.variables[v], 'units', s['units'][i])
    setattr(snow.variables[v], 'description', s['description'][i])


In [ ]:
# =======================================================================
# Get all files in the directory, open ipw file, and add to netCDF
# =======================================================================

# get all the files in the directory
d = sorted(glob("{}/snow*".format(out_dir)),
            key=os.path.getmtime)

d.sort(key=lambda f: os.path.splitext(f))

# find a drop any netcdfs in directory
d = [ddp for ddp in d if '.nc' not in ddp]
j = 0

for f in d:
    if j == int(len(d)/4):
        print("25 percent finished with "
                            "making NetCDF files!")
    if j == int(len(d)/2):
        print("50 percent finished with "
                            "making NetCDF files!")
    if j == int(3*len(d)/4):
        print("75 percent finished with "
                            "making NetCDF files!")
                            
    # get the hr
    nm = os.path.basename(f)
    head = os.path.dirname(f)
    hr = int(nm.split('.')[1])
    # hr = int(hr)
    snow.variables['time'][j] = hr  # +1
    em.variables['time'][j] = hr  # +1

    # Read the IPW file
    i = IPW(f)

    # output to the snow netcdf file
    for b, var in enumerate(s['name']):
        snow.variables[var][j, :] = i.bands[b].data

    # output to the em netcdf file
    # emFile = "%s/%s.%04i" % (head, 'em', hr)
    emFile = os.path.join(head, 'em.%04i' % (hr))
    i_em = IPW(emFile)
    for b, var in enumerate(m['name']):
        em.variables[var][j, :] = i_em.bands[b].data

    em.sync()
    snow.sync()
    j += 1

snow.close()
em.close()


In [ ]:
# convert the input ipw images to netcdf
bands = [
    'thermal',
    'air_temp',
    'vapor_pressure',
    'wind_speed',
    'soil_temp',
    'net_solar'
]

path_input = '../pysnobal/tests/test_data_spatial/gold_ipw/rme_wy2004/input'
num_files = len(glob(os.path.join(path_input, 'in.*')))

# create the netcdf files
files = {}
for band in bands:
    netcdfFile = os.path.join(path_output, '{}.nc'.format(band))
    os.remove(netcdfFile)
    print(netcdfFile)

    dimensions = ('time', 'y', 'x')
    input = nc.Dataset(netcdfFile, 'w', clobber=True)

    # create the dimensions
    input.createDimension('time', None)
    input.createDimension('y', ny)
    input.createDimension('x', nx)

    # create some variables
    input.createVariable('time', 'f', dimensions[0])
    input.createVariable('y', 'f', dimensions[1])
    input.createVariable('x', 'f', dimensions[2])

    setattr(input.variables['time'], 'units',
            'hours since %s' % start_date)
    setattr(input.variables['time'], 'calendar', 'standard')
    setattr(input.variables['time'], 'time_zone', time_zone)
    input.variables['x'][:] = x
    input.variables['y'][:] = y

    input.createVariable(band, 'f', dimensions)

    files[band] = input

for index in range(num_files-1):
    file_name = os.path.join(path_input, 'in.%04i' % (index))

    input_file = IPW(file_name)
    for j, band in enumerate(bands):

        # when the sun is down there will be one less band
        if j < len(input_file.bands):
            files[band].variables[band][index, :] = input_file.bands[j].data

for band in bands:
    files[band].close()
    

In [ ]:
# convert the precip ipw images to netcdf
bands = [
    'precip',
    'percent_snow',
    'snow_density',
    'precip_temp'
]

path_input = '../pysnobal/tests/test_data_spatial/gold_ipw/rme_wy2004/ppt'
num_files = len(glob(os.path.join(path_input, 'ppt.*')))

# create the netcdf files
files = {}
for band in bands:
    netcdfFile = os.path.join(path_output, '{}.nc'.format(band))
    os.remove(netcdfFile)
    print(netcdfFile)

    dimensions = ('time', 'y', 'x')
    input = nc.Dataset(netcdfFile, 'w', clobber=True)

    # create the dimensions
    input.createDimension('time', None)
    input.createDimension('y', ny)
    input.createDimension('x', nx)

    # create some variables
    input.createVariable('time', 'f', dimensions[0])
    input.createVariable('y', 'f', dimensions[1])
    input.createVariable('x', 'f', dimensions[2])

    setattr(input.variables['time'], 'units',
            'hours since %s' % start_date)
    setattr(input.variables['time'], 'calendar', 'standard')
    setattr(input.variables['time'], 'time_zone', time_zone)
    input.variables['x'][:] = x
    input.variables['y'][:] = y

    input.createVariable(band, 'f', dimensions)

    files[band] = input

for index in range(num_files-1):
    file_name = os.path.join(path_input, 'in.%04i' % (index))

    input_file = IPW(file_name)
    for j, band in enumerate(bands):

        # when the sun is down there will be one less band
        if j < len(input_file.bands):
            files[band].variables[band][index, :] = input_file.bands[j].data

for band in bands:
    files[band].close()

In [ ]:
snow = nc.Dataset(os.path.join(path_output, 'air_temp.nc'))

variable = 'air_temp'
index = 1200

plt.imshow(snow.variables[variable][index, :])
plt.colorbar()